<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">

# Accelerating End-to-End Data Science Workflows # 

## 02 - การเตรียมข้อมูลสำหรับการสร้างกราฟ ##

**สารบัญ**

โน้ตบุ๊กนี้จะแนะนำพื้นฐานของการแทนและสร้างกราฟ โน้ตบุ๊กนี้ครอบคลุมส่วนต่างๆ ดังนี้:

1.  [พื้นหลัง (Background)](#Background)
2.  [สภาพแวดล้อม (Environment)](#Environment)
3.  [อ่านข้อมูล (Read Data)](#Read-Data)
    * [โหนดถนนในสหราชอาณาจักร (UK Road Nodes)](#UK-Road-Nodes)
    * [ขอบถนนในสหราชอาณาจักร (UK Road Edges)](#UK-Road-Edges)
    * [แบบฝึกหัดที่ 1 - ทำให้ ID เข้ากันได้ (Make IDs Compatible)](#Exercise-#1---Make-IDs-Compatible)
4.  [สรุปข้อมูล (Data Summary)](#Data-Summary)
5.  [การสร้างกราฟเครือข่ายถนน (Building the Road Network Graph)](#Building-the-Road-Network-Graph)
    * [จัดเรียงดัชนี `road_nodes` ใหม่ (Reindex `road_nodes`)](#Reindex-road_nodes)
    * [การวิเคราะห์กราฟ (Analyzing the Graph)](#Analyzing-the-Graph)
6.  [สร้างกราฟถนนด้วยน้ำหนักเวลา (Construct a Graph of Roads with Time Weights)](#Construct-a-Graph-of-Roads-with-Time-Weights)
    * [การแปลงประเภทถนนเป็นความเร็ว (Road Type to Speed Conversion)](#Road-Type-to-Speed-Conversion)
    * [ขั้นตอนที่ 1: รวม `speed_gdf` เข้ากับ `road_edges` (Merge `speed_gdf` into `road_edges`)](#Step-1:-Merge-speed_gdf-into-road_edges)
    * [แบบฝึกหัดที่ 2 - ขั้นตอนที่ 2: เพิ่มคอลัมน์ความยาวเป็นวินาที (Add Length in Seconds Column)](#Exercse-#2---Step-2:-Add-Length-in-Seconds-Column)
    * [แบบฝึกหัดที่ 3 - ขั้นตอนที่ 3: สร้างกราฟ (Construct the Graph)](#Exercise-#3---Step-3:-Construct-the-Graph)

## Background ##
ส่วนหนึ่งของเป้าหมายด้านวิทยาศาสตร์ข้อมูลที่ใหญ่ขึ้นสำหรับเวิร์กช็อปนี้ เราจะทำงานกับข้อมูลที่สะท้อนถึงเครือข่ายถนนทั้งหมดของบริเตนใหญ่ เรามีข้อมูลถนนที่แยกออกมาในรูปแบบ CSV แบบตารางจากไฟล์ [GML](https://en.wikipedia.org/wiki/Geography_Markup_Language) อย่างเป็นทางการเป็นจุดเริ่มต้น สุดท้ายนี้ เราต้องการใช้ cuGraph เพื่อทำการวิเคราะห์กราฟที่เร่งความเร็วด้วย GPU บนข้อมูลนี้ แต่ในการทำเช่นนั้น เราจำเป็นต้องทำการประมวลผลล่วงหน้าบางอย่างเพื่อเตรียมข้อมูลให้พร้อมสำหรับการสร้างกราฟ

ในโน้ตบุ๊กนี้ คุณจะได้เรียนรู้เทคนิคการแปลงข้อมูล cuDF เพิ่มเติมในการสาธิตการเตรียมข้อมูลสำหรับการนำเข้าโดย cuGraph ถัดไป คุณจะได้ทำแบบฝึกหัดชุดหนึ่งเพื่อทำการแปลงข้อมูลที่คล้ายกันสำหรับการสร้างกราฟที่มีน้ำหนักขอบที่แตกต่างกัน








## Environment ##

นอกเหนือจาก `cudf` แล้ว ในโน้ตบุ๊กนี้เราจะนำเข้า `cugraph` ซึ่งเราจะใช้ (หลังจากเตรียมข้อมูลแล้ว) เพื่อสร้างกราฟที่เร่งความเร็วด้วย GPU เรายังนำเข้า `networkx` สำหรับการเปรียบเทียบประสิทธิภาพสั้นๆ ในภายหลัง

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import cudf
import cugraph as cg

import networkx as nx

## Read Data ##

ในหน้านี้ (notebook) เราจะใช้ข้อมูล 2 แหล่ง เพื่อสร้างกราฟแสดงเครือข่ายถนนของสหราชอาณาจักร (UK) ครับ

### UK Road Nodes ###

ตารางข้อมูลแรกจะอธิบายถึง **จุดเชื่อม (nodes)** ในเครือข่ายถนน ได้แก่ **จุดปลายทาง (endpoints)**, **ทางแยก (junctions)** (รวมถึงวงเวียน), และ **จุดที่แบ่งถนนโค้งยาวๆ** เพื่อให้สามารถสร้างแผนที่ได้อย่างถูกต้อง (แทนที่จะเป็นเส้นตรง)

พิกัดสำหรับแต่ละจุดอยู่ในรูปแบบ **OSGB36** ซึ่งเราได้ศึกษาไปแล้วในหัวข้อ 1-05


In [ ]:
road_nodes = cudf.read_csv('./data/road_nodes.csv')
road_nodes.head()

In [ ]:
road_nodes.dtypes

In [ ]:
road_nodes.shape

In [ ]:
road_nodes['type'].unique()

### UK Road Edges ###

ตารางข้อมูลที่สองจะอธิบายถึง **ส่วนของถนน (road segments)** ซึ่งรวมถึง **จุดเริ่มต้นและจุดสิ้นสุด (start and end points)**, **ความยาว (how long they are)**, และ **ประเภทของถนน (what kind of road they are)** นั้น ๆ ครับ

In [ ]:
road_edges = cudf.read_csv('./data/road_edges.csv')
road_edges.head()

In [ ]:
road_edges.dtypes

In [ ]:
road_edges.shape

In [ ]:
road_edges['type'].unique()

In [ ]:
road_edges['form'].unique()

### Exercise #1 - Make IDs Compatible ###

ไฟล์ CSV ของเราได้มาจากไฟล์ [GML](https://en.wikipedia.org/wiki/Geography_Markup_Language) ต้นฉบับ และอย่างที่คุณเห็นจากข้อมูลข้างต้น ทั้ง `road_edges['src_id']` และ `road_edges['dst_id']` มีอักขระ `#` นำหน้า ซึ่ง `road_nodes['node_id']` ไม่มี เพื่อให้ ID สามารถใช้งานร่วมกันได้ระหว่าง edges (เส้นเชื่อม) และ nodes (จุดเชื่อม), ให้ใช้ **เมธอดสตริง** ของ cuDF คือ `.str.lstrip` เพื่อแทนที่คอลัมน์ `src_id` และ `dst_id` ใน `road_edges` ด้วยค่าที่ถูกลบอักขระ `#` ที่นำหน้าออกไปแล้ว


Click ... for solution. 

## Data Summary ##

เมื่อข้อมูลได้รับการทำความสะอาดแล้ว เราจะเห็นจำนวนถนนและจุดปลายทาง/ทางแยก/จุดโค้งที่เราจะใช้งาน รวมถึงปริมาณการใช้หน่วยความจำบน GPU ของเราด้วย GPU ที่เราใช้อยู่สามารถจัดเก็บและวิเคราะห์กราฟที่ใหญ่กว่านี้ได้มาก!

In [ ]:
print(f'{road_edges.shape[0]} edges, {road_nodes.shape[0]} nodes')

In [ ]:
!nvidia-smi

## Building the Road Network Graph ##

เราไม่มีข้อมูลทิศทางการจราจรของถนน (บางเส้นเป็นเดินรถทางเดียว) ดังนั้น **เพื่อความง่าย** เราจะถือว่าถนนทุกสายเป็นแบบ **สองทาง (two-way)** ทั้งหมด การทำเช่นนี้ทำให้กราฟของเราเป็น **"undirected" (ไม่มีทิศทาง)** ดังนั้นเราจะสร้าง **cuGraph `Graph`** แทนที่จะเป็น directed graph หรือ `DiGraph` ครับ

เราจะเริ่มต้น (initialize) กราฟด้วยข้อมูล **แหล่งกำเนิด (edge sources)**, **ปลายทาง (destinations)**, และ **คุณสมบัติ (attributes)** ซึ่งในข้อมูลของเราก็คือ **ความยาวของถนน (length of the roads)** นั่นเองครับ

In [ ]:
G = cg.Graph()
%time G.from_cudf_edgelist(road_edges, source='src_id', destination='dst_id', edge_attr='length')

เพื่อใช้เปรียบเทียบ เราจะสร้างกราฟที่เทียบเท่ากันใน **NetworkX** จาก Pandas dataframe ที่ผ่านการทำความสะอาดและเตรียมข้อมูลมาแล้วด้วยครับ

In [ ]:
road_edges_cpu = road_edges.to_pandas()
%time G_cpu = nx.convert_matrix.from_pandas_edgelist(road_edges_cpu, source='src_id', target='dst_id', edge_attr='length')

### Reindex `road_nodes` ###

เพื่อการค้นหาที่มีประสิทธิภาพในภายหลัง เราจะ **จัดเรียงดัชนี (reindex)** ของ `road_nodes` ใหม่ให้ใช้ **`node_id` เป็นดัชนี** ของมัน — จำไว้ว่าโดยทั่วไปแล้วเราจะได้ผลลัพธ์จากการวิเคราะห์กราฟในรูปของ `node_id` ดังนั้นวิธีนี้จะช่วยให้เราดึงข้อมูลอื่น ๆ เกี่ยวกับโหนด (เช่น ตำแหน่ง) ได้อย่างง่ายดาย จากนั้นเราจะ **เรียงลำดับ (sort)** DataFrame ตามดัชนีใหม่นี้

In [ ]:
road_nodes = road_nodes.set_index('node_id', drop=True)
%time road_nodes = road_nodes.sort_index()
road_nodes.head()

### Analyzing the Graph ###

เมื่อเราสร้างกราฟเสร็จแล้ว เราสามารถวิเคราะห์**จำนวนโหนด (nodes)** และ**เส้นเชื่อม (edges)** ในกราฟได้ดังนี้:

In [ ]:
G.number_of_nodes()

In [ ]:
G.number_of_edges()

ข้อสังเกตเกี่ยวกับการลดจำนวน Edge

จะสังเกตเห็นว่าจำนวน **edges (เส้นเชื่อม)** มีจำนวนน้อยกว่าจำนวน edges ใน `road_edges` ที่แสดงไว้ข้างต้นเล็กน้อย นั่นเป็นเพราะข้อมูลต้นฉบับมาจาก **map tiles (แผ่นแผนที่)** และถนนที่พาดผ่านขอบของแผ่นแผนที่นั้นจะถูกแสดงอยู่ในทั้งสองแผ่น ทำให้ cuGraph ทำการ **กำจัดข้อมูลซ้ำซ้อน (de-duplicated)** ออกไป หากเรากำลังสร้าง `MultiGraph` หรือ `MultiDiGraph` ซึ่งเป็นกราฟที่สามารถมีหลายเส้นเชื่อมไปในทิศทางเดียวกันระหว่างจุดเชื่อมได้ ข้อมูลที่ซ้ำซ้อนกันนั้นก็จะถูกเก็บรักษาไว้








เราสามารถวิเคราะห์ degrees ของ node ในกราฟของเราได้ด้วย:

In [ ]:
deg_df = G.degree()

ในกราฟที่ **ไม่มีทิศทาง (undirected graph)** เส้นเชื่อม (edge) ทุกเส้นที่ **เข้าสู่ (entering)** โหนด (node) จะถือเป็นเส้นเชื่อมที่ **ออกจาก (leaving)** โหนดนั้นไปพร้อมกัน ดังนั้นเราจึงคาดว่าโหนดควรมี **ดีกรี (degree)** อย่างน้อย 2 ครับ

In [ ]:
deg_df['degree'].describe()[1:]

คุณจะได้ใช้กราฟที่ประมวลผลด้วย GPU นี้มากขึ้นในภายหลังของเวิร์คช็อปครับ

## Construct a Graph of Roads with Time Weights ##

สำหรับแบบฝึกหัดชุดนี้ คุณจะได้สร้างและวิเคราะห์กราฟใหม่ของถนนในประเทศอังกฤษ (Great Britain) โดยใช้เทคนิคที่เพิ่งสาธิตไป แต่คราวนี้ แทนที่จะใช้ระยะทางดิบเป็นค่าน้ำหนักของเส้นเชื่อม (edges' weights) คุณจะใช้ **เวลาที่ใช้ในการเดินทาง** ระหว่างจุดเชื่อม (nodes) สองจุด โดยอ้างอิงจากความเร็วจำกัดที่กำหนด

คุณจะเริ่มแบบฝึกหัดนี้ด้วย DataFrame `road_edges` ที่ใช้ไปก่อนหน้านี้

In [ ]:
road_edges.dtypes

### Road Type to Speed Conversion ###

เพื่อที่จะคำนวณว่าควรใช้เวลาเดินทางบนถนนเส้นหนึ่งนานเท่าใด เราจำเป็นต้องทราบ **ขีดจำกัดความเร็ว (speed limit)** ของถนนนั้น เราจะทำสิ่งนี้โดยใช้ประโยชน์จาก `road_edges['type']` พร้อมกับกฎสำหรับขีดจำกัดความเร็วของถนนแต่ละประเภท

นี่คือประเภทถนนที่ไม่ซ้ำกันในข้อมูลของเรา:

In [ ]:
road_edges['type'].unique()

และนี่คือตารางที่แสดง **ข้อสมมติฐานเกี่ยวกับความเร็วสูงสุด (speed limits)** ที่เราจะใช้สำหรับการแปลงข้อมูลของเราครับ:

In [ ]:
# https://www.rac.co.uk/drive/advice/legal/speed-limits/
# Technically, speed limits depend on whether a road is in a built-up area and the form of carriageway,
# but we can use road type as a proxy for built-up areas.
# Values are in mph.

speed_limits = {'Motorway': 70,
               'A Road': 60,
               'B Road': 60,
               'Local Road': 30,
               'Local Access Road': 30,
               'Minor Road': 30,
               'Restricted Local Access Road': 30,
               'Secondary Access Road': 30}

เรามาเริ่มต้นด้วยการสร้าง `speed_gdf` เพื่อเก็บข้อมูล **ประเภทถนน** แต่ละชนิดพร้อมกับ **การจำกัดความเร็ว** ของมันครับ

In [ ]:
speed_gdf = cudf.DataFrame()

speed_gdf['type'] = speed_limits.keys()
speed_gdf['limit_mph'] = [speed_limits[key] for key in speed_limits.keys()]
speed_gdf

ถัดไป เราจะเพิ่มคอลัมน์ใหม่ชื่อ `limit_m/s` ซึ่งจะบอก **ความเร็วสูงสุดที่สามารถเดินทางบนถนนแต่ละประเภทได้ในหน่วยเมตรต่อวินาที**

In [ ]:
# We will have road distances in meters (m), so to get road distances in seconds (s), we need to multiply by meters/mile and divide by seconds/hour
# 1 mile ~ 1609.34 m
speed_gdf['limit_m/s'] = speed_gdf['limit_mph'] * 1609.34 / 3600
speed_gdf

### Step 1: Merge `speed_gdf` into `road_edges` ###

cuDF มีฟังก์ชันการรวมข้อมูลเหมือนกับ Pandas เลยครับ เนื่องจากเราจะใช้ค่าใน `road_edges` เพื่อสร้างกราฟของเรา เราจึงจำเป็นต้อง **รวม (merge)** `speed_gdf` เข้ากับ `road_edges` (คล้ายกับการ join ในฐานข้อมูล) คุณสามารถรวมข้อมูลโดยใช้คอลัมน์ `type` ซึ่งเป็นคอลัมน์ที่มีร่วมกันในทั้งสอง DataFrame นี้ครับ

In [ ]:
%time road_edges = road_edges.merge(speed_gdf, on='type')

### Exercse #2 - Step 2: Add Length in Seconds Column ###

คุณต้องคำนวณจำนวนวินาทีที่จะใช้ในการเดินทางบนถนนที่กำหนดด้วยความเร็วสูงสุดที่อนุญาต สามารถทำได้โดยการนำ **ความยาวของถนนในหน่วยเมตร (m)** หารด้วย **ความเร็วสูงสุดในหน่วยเมตรต่อวินาที (m/s)** ทำการคำนวณนี้กับ `road_edges` และเก็บผลลัพธ์ไว้ในคอลัมน์ใหม่ชื่อ `length_s`

Click ... for solution. 

### Exercise #3 - Step 3: Construct the Graph ###

สร้าง `Graph` ของ cuGraph ที่ชื่อว่า `G_ex` โดยใช้จุดเริ่มต้นและจุดสิ้นสุดที่อยู่ใน `road_edges` พร้อมกับค่าน้ำหนักของเส้นเชื่อม (edges) เป็นความยาวในหน่วยวินาที

Click ... for solution. 

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**ทำได้ดีมาก!** ไปยัง [สมุดบันทึกถัดไป](2-03_cugraph.ipynb) กัน